In [9]:
import pandas as pd
import pymongo
import json

In [10]:
mng_client = pymongo.MongoClient('localhost', 27017)
# Replace mongo db name
mng_db = mng_client['mydb']
#  Replace mongo db collection name
collection_name = 'sample' 
db_cm = mng_db[collection_name]

In [11]:
def insertDocument(doc, targetCollections):
    cursor = db_cm.find( {}, { "_id": 1 } ).sort("_id", -1).limit(1)
    empty = False
    try:
        record = cursor.next()
    except StopIteration:
        print("Warning! NO PREVIOUS RECORD, THIS IS THE FIRST TIME INSERTING ")
        empty = True
    if empty:
        seq = 1
    else:
        seq = record["_id"]+1
    doc["_id"] = seq
    results = targetCollections.insert(doc)

In [12]:
filename = "master_metadata_20171031.csv"
df = pd.read_csv(filename,sep=None,engine='python')

/home/ubuntu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  from ipykernel import kernelapp as app


In [13]:
# there is an extra field with the same name library_source, if bojan delete that field, I need to change this code
# df = df.drop('library_source.1', axis=1)

In [14]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [15]:
df['ir_sequence_count'] = 0

In [16]:
records = json.loads(df.T.to_json()).values()
record_list = list(records)
# db_cm.insert(records)
for r in record_list:
    insertDocument(r,db_cm)

Warning! NO PREVIOUS RECORD, THIS IS THE FIRST TIME INSERTING 


/home/ubuntu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:14: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
